In [ ]:
import os
import sys
import geopandas as gpd
import pandas as pd
from pathlib import Path

project_dir = Path().resolve().parent.parent
sys.path.append(str(project_dir))

local_crs = 32636

files_base_dir = Path(...)  # insert path to geojsons directory here
no_development_pzz_file = files_base_dir / "no_development_pzz.geojson"
landuse_zone_pzz_file = files_base_dir / "landuse_zone_pzz.geojson"
petropavlovsraya_file = files_base_dir / "petropavlovsraya_delete.geojson"

In [ ]:
from DataGetter_example import DataGetter as dg

dg = dg(city_crs=local_crs)

In [ ]:
city_admin_level = 5
city_name = "Санкт-Петербург"
buffer = 5
cutoff_area = 10_000

water = dg.get_water_geometry(city_name=city_name, water_buffer=buffer)
city_geom = dg.get_city_geometry(city_name=city_name, city_admin_level=city_admin_level)
nature = dg.get_nature_geometry(city_name=city_name, nature_buffer=buffer, park_cutoff_area=cutoff_area)
rails = dg.get_railways_geometry(city_name=city_name, railways_buffer=buffer)
roads = dg.get_roads_geometry(city_geometry=city_geom, roads_buffer=buffer)

In [ ]:
no_dev_zone = gpd.read_file(no_development_pzz_file)
no_dev_zone = no_dev_zone.to_crs(local_crs)
landuse_zone = gpd.read_file(landuse_zone_pzz_file)
landuse_zone = landuse_zone.to_crs(local_crs)

In [ ]:
petropavlovskaya = gpd.read_file(petropavlovsraya_file)
petropavlovskaya = petropavlovskaya.to_crs(local_crs)

In [ ]:
no_dev_zone = pd.concat([no_dev_zone[["geometry"]], petropavlovskaya[["geometry"]]], axis=0)

In [ ]:
from masterplan_tools.City_model.city_model import CityModel

cm = CityModel(
    water_geometry=water,
    roads_geometry=roads,
    railways_geometry=rails,
    nature_geometry_boundaries=nature,
    city_geometry=city_geom,
    landuse_zone=landuse_zone,
    no_dev_zone=no_dev_zone,
)

In [ ]:
from masterplan_tools.Blocks_getter import BlocksCutter

blocks = BlocksCutter(city_model=cm).get_blocks()

In [ ]:
from masterplan_tools.Blocks_getter import LuFilter

blocks = LuFilter(blocks).filter_lu()

In [ ]:
from masterplan_tools.Blocks_getter import BlocksClusterization

buildings_geom = gpd.read_feather("/home/gk/code/blocks_ysc/spb_buildings_geom.feather")
blocks = BlocksClusterization(blocks, buildings_geom).run()
blocks = blocks[blocks["landuse"].isin(["new_buildings", "buildings"])]